# Restricted Boltzmann machine 

In [ ]:
A00=dlmread('char_learn03.txt');

# 5 times Input data are made by adding noises
A0=0.8*A00+0.1;
A=vertcat(A0,A0,A0,A0,A0);
[n N]=size(A);
A=A+0.1*randn(n,N);
[n0 Ntest]=size(A0);

# n = number of input vectors
# N = dimension of input

YOKO=5;   # Character size 
CHARN=n0;  # Number of Input Patterns
H=8;     # hidden variables
ETA=0.01; # Coefficient of learning 
CYCLE=500; # cycle of learning 
MCMC=20;  # MCMC process 1 5 20 50 100
MCMCTEST=50; # MCMC for TEST
NSEE=10;  # Visible units in testy 5 10 15 20 25 
NOISETEST=0.1;

# sigmoid function 
sigmoid=@(t)(1./(1+exp(-t)));

# Initial parameters
w=0.2*randn(H,N);
th1=0.2*randn(H,1);
th2=0.2*randn(N,1);

for cycle=1:1:CYCLE
 for i=1:1:n
  hid_given=sigmoid(w*A(i,:)'+th1);
  S=0.5*ones(H+N,1);
  SUMS=zeros(H+N,1);
  COVS=zeros(H+N,H+N);
  for mcmc=1:1:MCMC
   S(1:1:H,1)=floor(sigmoid(w*S(H+1:1:H+N,1)+th1)+rand(H,1));
   S(H+1:1:H+N,1)=floor(sigmoid(w'*S(1:1:H,1)+th2)+rand(N,1));
   SUMS=SUMS+S;
   COVS=COVS+S*S';
  end
  SUMS=SUMS/MCMC;
  COVS=COVS/MCMC;
  w=w+ETA*(hid_given*A(i,:)-COVS(1:1:H,H+1:1:H+N));
  th1=th1+ETA*(hid_given-SUMS(1:1:H,1));
  th2=th2+ETA*(A(i,:)'-SUMS(H+1:1:H+N,1));
 end
end

# Trained DATA
xx=zeros(6,5);
for i=1:1:n0
 subplot(3,CHARN,i);
 for j=1:1:N
  xx(floor((j-1)/YOKO)+1,mod(j-1,YOKO)+1)=A(i,j);
 end
 image(255*(1-0.5*xx)); hold on;
 colormap(gray(255));
 axis off;
 axis ij;
end

# TEST DATA 

TESTA=A00+NOISETEST*randn(n0,N);
for i=1:1:n0
 for j=(NSEE+1):1:N
   TESTA(i,j)=0.5;
 end
subplot(3,CHARN,CHARN+i);
 for j=1:1:N
  xx(floor((j-1)/YOKO)+1,mod(j-1,YOKO)+1)=TESTA(i,j);
 end
 image(255*(1-0.5*xx)); hold on;
 colormap(gray(255));
 axis off;
 axis ij;
end

# Results of Test data
fprintf('Hidden Unit Answer for Test\n');
for i=1:1:n0
  S=0.5*ones(H+N,1);
  SUMS=zeros(H+N,1);
  for mcmc=1:1:MCMCTEST
   S(H+1:1:H+NSEE,1)=TESTA(i,1:1:NSEE)';
   S(1:1:H,1)=floor(sigmoid(w*S(H+1:1:H+N,1)+th1)+rand(H,1));
   S(H+1:1:H+N,1)=floor(sigmoid(w'*S(1:1:H,1)+th2)+rand(N,1));
   SUMS=SUMS+S;
  end
 subplot(3,CHARN,2*CHARN+i);
 fprintf('Pattern %g: ',i);
  for ii=1:1:H
    fprintf('%.2f ',SUMS(ii,1)/MCMCTEST);
  end
  fprintf('\n');
 for j=1:1:N
  xx(floor((j-1)/YOKO)+1,mod(j-1,YOKO)+1)=SUMS(H+j,1)/MCMCTEST;
 end
 image(255*(1-0.5*xx)); hold on;
 colormap(gray(255));
 axis off;
 axis ij;
end
